In [9]:
from utilities import *
from credentials import *
from langchain_openai import ChatOpenAI


model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, 
                       openai_api_base=OPENAI_BASE_URL, 
                       model=OPENAI_MODEL_NAME,
                       verbose = True,
                       temperature=0)

db_connection = psycopg2.connect(
            dbname=POSTGRES_DB,
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_HOST,
            port=POSTGRES_PORT
        )

question = "Whats a good restaurant near NY Aquarium?"
pdf_name = "sightseeing_pass_sites"
cssv_name = "google_maps_restaurants_shortened_2"


In [19]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

def build_retrieval_query(pdf_file_name):
    return f"""
        WITH node, score AS closestScore
        ORDER BY closestScore DESC
        LIMIT 1

        WITH node, closestScore,
             node.blockId     AS nClosestScoreBlockId,
             node.blockNumber AS nClosestScoreBlockNumber,
             node.chunkSeqIndex  AS nClosestScoreChunk,
             node.text        AS nClosestScoreText,
             node.source      AS nClosestScoreSource,
             node.pageNumber  AS nClosestScorePageNumber,
             node.isHeader    AS nClosestScoreIsHeader

        MATCH (anyPage:Page)
        WHERE anyPage.pdfFileName = '{pdf_file_name}'
        WITH node, closestScore, nClosestScorePageNumber, nClosestScoreIsHeader, nClosestScoreBlockId, nClosestScoreText, nClosestScoreBlockNumber,
             count(anyPage) AS totalPages


        MATCH (node:Block) 
        OPTIONAL MATCH (prev:Block)-[:NEXT*0..]->(node)
        WHERE prev.pdfFileName = '{pdf_file_name}'
          AND prev.pageNumber = nClosestScorePageNumber
          AND prev.blockNumber < nClosestScoreBlockNumber
          AND prev.isHeader = true
        ORDER BY prev.blockNumber DESC
        LIMIT 1
        OPTIONAL MATCH (node)-[:NEXT*0..]->(next:Block)
        WHERE next.pdfFileName = '{pdf_file_name}'
          AND next.pageNumber = nClosestScorePageNumber
          AND next.blockNumber > nClosestScoreBlockNumber
          AND next.isHeader = true
        ORDER BY next.blockNumber ASC
        LIMIT 1

        OPTIONAL MATCH p = (prev)-[:NEXT*]->(next)
        WHERE ALL(n IN nodes(p) WHERE n.pageNumber = nClosestScorePageNumber)
        WITH node, closestScore, nClosestScorePageNumber, nClosestScoreIsHeader, nClosestScoreBlockId, 
             nClosestScoreText, nClosestScoreBlockNumber, totalPages, prev, next,
             COALESCE(nodes(p)[1..-1], []) AS blocksBetween

        OPTIONAL MATCH q = (b: Block)-[:NEXT*]->(next)
        WHERE ALL(n IN nodes(q) WHERE n.pageNumber = nClosestScorePageNumber AND n.blockNumber >= 0)
        WITH node, closestScore, nClosestScorePageNumber, nClosestScoreIsHeader, nClosestScoreBlockId, 
             nClosestScoreText, nClosestScoreBlockNumber, totalPages, prev, next, blocksBetween,
             COALESCE(nodes(q)[1..-1], []) AS startToNext

        OPTIONAL MATCH r = (prev)-[:NEXT*]->(b: Block)
        WHERE ALL(n IN nodes(r) WHERE n.pageNumber = nClosestScorePageNumber AND n.blockNumber <= totalPages)
        WITH node, closestScore, nClosestScorePageNumber, nClosestScoreIsHeader, nClosestScoreBlockId, 
             nClosestScoreText, nClosestScoreBlockNumber, totalPages, prev, next, blocksBetween, startToNext,
             COALESCE(nodes(r)[1..-1], []) AS prevToEnd
             
        WITH node, closestScore, nClosestScorePageNumber, nClosestScoreIsHeader, nClosestScoreBlockId, 
             nClosestScoreText, nClosestScoreBlockNumber, totalPages, prev, next, blocksBetween, startToNext, prevToEnd,
             CASE
                 WHEN prev IS NOT NULL AND next IS NOT NULL THEN blocksBetween
                 WHEN prev IS NOT NULL AND next IS NULL THEN prevToEnd
                 ELSE startToNext
             END AS finalWindow

        
        RETURN
          // If finalWindow is an array, join the texts; adjust as needed:
          apoc.text.join([x IN finalWindow | x.text], " ") AS text,
          closestScore AS score,
          node {{
            source: node.pageNumber,
            text: node.text,
            next: next.text,
            prev: prev.text
          }} AS metadata
    """



def kg_retrieval_window(file_name):
    """retrieval window for the knowledge graph"""
    vector_store_window = Neo4jVector.from_existing_index(
        embedding=OpenAIEmbeddings(
            openai_api_key=OPENAI_API_KEY,
            openai_api_base=OPENAI_BASE_URL,
            model=OPENAI_EMB_MODEL_SMALL,
            dimensions=512
        ),
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        database=NEO4J_DATABASE,
        index_name="pdf_lines",
        text_node_property='text',
        retrieval_query= build_retrieval_query(file_name)
    )
    retriever_window = vector_store_window.as_retriever()

    return retriever_window


retriever = kg_retrieval_window("sightseeing_pass_sites")
question = "We provide subway or walking directions after the tour"
retrieval_results = retriever.invoke(question)
rprint(retrieval_results)
    

[
    Document(
        metadata={
            'text': 'confirmation. We provide subway or walking directions after the tour.',
            'source': 12,
            'next': 'What You Get: The Sightseeing Pass grants you admission to Go West Guided Public Food Tour. 
Tour meets near 8th Av. at W. 50th St. Specific meeting point is on your confirmation. No subway fare is required 
for this tour.'
        },
        page_content=''
    ),
    Document(
        metadata={
            'text': 'confirmation. We provide subway or walking directions after the tour.',
            'source': 12,
            'next': 'What You Get: The Sightseeing Pass grants you admission to Go West Guided Public Food Tour. 
Tour meets near 8th Av. at W. 50th St. Specific meeting point is on your confirmation. No subway fare is required 
for this tour.'
        },
        page_content='are posted on or before the 16th of the previous month but usually earlier. You will be asked
to provide a credit card and your pass numbers and/or order confirmation number for security. Click the pass holder
booking link at www.foodonfoottours.com to secure tickets.'
    ),
    Document(
        metadata={
            'text': 'confirmation. We provide subway or walking directions after the tour.',
            'source': 12,
            'next': 'What You Get: The Sightseeing Pass grants you admission to Go West Guided Public Food Tour. 
Tour meets near 8th Av. at W. 50th St. Specific meeting point is on your confirmation. No subway fare is required 
for this tour.'
        },
        page_content='summary/How To Book, About & Rates. Use code NYSFoodfun to order your tickets. Food On Foot 
Tours often sell out. Advance booking before you arrive in NY is recommended. If a tour you would like is sold out 
please check the calendar the day before or morning of as we sometimes receive cancellations. Tours do not run 
every day. Please note: Guided tour schedules are posted on or before the 16th of are posted on or before the 16th 
of the previous month but usually earlier. You will be asked to provide a credit card and your pass numbers and/or 
order confirmation number for security. Click the pass holder booking link at www.foodonfoottours.com to secure 
tickets.'
    )
]

In [ ]:
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain


graph = Neo4jGraph(
    url=NEO4J_URI,  # or your hosted DB
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# print(graph.schema)

chain = GraphCypherQAChain.from_llm(
                model,
                allow_dangerous_requests=True, graph=graph, verbose=True
                )

# graph_result = chain.invoke("look for blocks that deal with the question subway token?")

# print(graph_result)

In [6]:
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


PDFAGENTPROMPTTEMPLATE_B = PromptTemplate(
    template=   """
                system,
                You are an information gatherer for a pdf document. Given the following extracted parts of a PDF document (called 'summaries'), a question and columns from a table (called 'columns') your task is:
                1. Gather any fundamental information that is relevant to the question (e.g., addresses, dates, values, events). 
                2. Look specifically for information related to each column in the table.
                3. Place all relevent information in the 'response' field including useful data points
                4. Place the data points from the pdf in the 'data_points' field without any other text. Use commas to separate data points and order the data points by most specific to least specific.
                5. Place all related column names in 'relevant_columns'. 
                6. Return a correctly formatted python dictionary

                question: {question}
                columns: {columns}
                =========
                {summaries}
                =========
                Respond using python dictionary:    "response" :  "<_START_> Your complete answer <_END_>", 
                                                    "data_points": Most relevant data points such as addresses, names, dates, emails, etc", 
                                                    "relevant_columns": Top 3 most relevant column names seperated by commas" 
                """, 
            input_variables=["summaries", "question", "columns"])



In [7]:
from langchain.chains import RetrievalQAWithSourcesChain


def kg_retrieval_chain(prompt, file_name):
    chain_type_kwargs = {"prompt": prompt}

    kg_chain_window = RetrievalQAWithSourcesChain.from_chain_type(
        model, 
        chain_type = "stuff", 
        retriever = kg_retrieval_window(file_name),
        chain_type_kwargs = chain_type_kwargs,
        return_source_documents = True
    )

    answer = kg_chain_window.invoke(input_variables, return_only_outputs=True)

    return answer


In [ ]:

columns = get_all_columns_and_types("google_maps_restaurants_shortened_2")
col_str = ", ".join(item[0] for item in columns)

input_variables={"question": quesstion, "columns": col_str, "pdf_name": pdf_name}

pdf_retreival = kg_retrieval_chain(PDFAGENTPROMPTTEMPLATE_B, csv_name)
rprint(pdf_retreival)

pdf_retreival_llm_response = pdf_retreival["answer"]
answer = convert_to_dict(pdf_retreival_llm_response)
pdf_answer = answer["response"]
pdf_data_points = answer["data_points"]
pdf_relevant_columns = answer["relevant_columns"]


{
    'answer': '{\n    "response": "<_START_> Unfortunately, the provided content does not include information about
restaurants near the NY Aquarium. The only detailed information available pertains to the NY Aquarium itself, 
including its address at 602 Surf Avenue, Brooklyn, New York 11224, operating hours, and directions on how to get 
there by subway. For recommendations on good restaurants near the NY Aquarium, additional sources would need to be 
consulted. <_END_>",\n    "data_points": "602 Surf Avenue, Brooklyn, New York 11224, 10:00am - 5:00pm, 10:00am - 
5:30pm, 800.433.4149",\n    "relevant_columns": "address, zipcode"\n}',
    'sources': '',
    'source_documents': [
        Document(
            metadata={
                'text': 'NY Aquarium',
                'source': 1,
                'prev': 'What You Get: The Sightseeing Pass grants you free admission to: Northlandz Miniature 
Wonderland'
            },
            page_content=''
        ),
        Document(
            metadata={
                'text': 'NY Aquarium',
                'source': 1,
                'prev': 'What You Get: The Sightseeing Pass grants you free admission to: Northlandz Miniature 
Wonderland'
            },
            page_content='NY Aquarium'
        ),
        Document(
            metadata={
                'text': 'NY Aquarium',
                'source': 1,
                'prev': 'What You Get: The Sightseeing Pass grants you free admission to: Northlandz Miniature 
Wonderland'
            },
            page_content='NY Aquarium 602 Surf Avenue, Brooklyn, New York 11224'
        ),
        Document(
            metadata={
                'text': 'NY Aquarium',
                'source': 1,
                'prev': 'What You Get: The Sightseeing Pass grants you free admission to: Northlandz Miniature 
Wonderland'
            },
            page_content='NY Aquarium 602 Surf Avenue, Brooklyn, New York 11224 Weekdays: 10:00am - 5:00pm Weekends
& Holidays: 10:00am - 5:30pm NOTE: Passes are not accepted on Wednesday afternoons. Aquatheater shows are weather 
dependent and may be cancelled due to rain.'
        ),
        Document(
            metadata={
                'text': 'NY Aquarium',
                'source': 1,
                'prev': 'What You Get: The Sightseeing Pass grants you free admission to: Northlandz Miniature 
Wonderland'
            },
            page_content="NY Aquarium 602 Surf Avenue, Brooklyn, New York 11224 Weekdays: 10:00am - 5:00pm Weekends
& Holidays: 10:00am - 5:30pm NOTE: Passes are not accepted on Wednesday afternoons. Aquatheater shows are weather 
dependent and may be cancelled due to rain. All admissions require timed ticket entry; you must reserve your timed 
tickets 24-48 hours prior to your visit by calling WCS's Group Sales Department at 800.433.4149. All booking 
availability is first come first serve."
        ),
        Document(
            metadata={
                'text': 'NY Aquarium',
                'source': 1,
                'prev': 'What You Get: The Sightseeing Pass grants you free admission to: Northlandz Miniature 
Wonderland'
            },
            page_content="NY Aquarium 602 Surf Avenue, Brooklyn, New York 11224 Weekdays: 10:00am - 5:00pm Weekends
& Holidays: 10:00am - 5:30pm NOTE: Passes are not accepted on Wednesday afternoons. Aquatheater shows are weather 
dependent and may be cancelled due to rain. All admissions require timed ticket entry; you must reserve your timed 
tickets 24-48 hours prior to your visit by calling WCS's Group Sales Department at 800.433.4149. All booking 
availability is first come first serve. Subway: Take the F or Q train to the West 8th Street station on Coney 
Island, Brooklyn. Alternatively, take the N or D trains to the Coney Island- Stillwell Avenue Station, then walk 
two blocks east on Surf Ave. (Stillwell Avenue station is handicap accessible on the F, Q, N, D train)"
        )
    ]
}

In [46]:

def levenshtein_dist_from_db(table_name: str, words: str):
    """ Counts how many single-character edits (insertion, deletion, substitution) it takes to transform one string into another.
        It has no understanding of meaning, context, or semantics — it’s purely syntactic."""

    try:
        connection = db_connection
        cur = connection.cursor()
    
        words_list = re.split(r"[,\s]+", words.strip())
    
        columns_and_types = get_all_columns_and_types_tuple(table_name)
        column_types = {col: dtype for col, dtype in columns_and_types}
    
        results = []

        for word in words_list:
            query_parts = []
    
            for column_name, _ in columns_and_types:
                query_parts.append(f"""
                    SELECT '{column_name}' AS column_name,
                           {column_name}::text AS column_value,
                           levenshtein({column_name}::text, %s) AS lev_distance
                    FROM {table_name}
                """)
    
            # Combine all column queries with UNION ALL
            final_query = " UNION ALL ".join(query_parts) + " ORDER BY lev_distance LIMIT 30;"
    
            cur.execute(final_query, (word,) * len(columns_and_types))
            rows = cur.fetchall()
            for row in rows:
                results.append((word, *row))
    
        seen = set()
        unique_results = []
        for result in results:
            if result not in seen:
                seen.add(result)
                unique_results.append(result)

        # rprint(unique_results)
        # Filter only non-numeric columns
        numeric_types = {"integer", "bigint", "smallint", "decimal", "numeric", "real", "double precision", "serial", "bigserial", "money"}
        filtered_results = [
            r for r in unique_results
            if column_types.get(r[1], "").lower() in numeric_types and r[3] <= 5
        ]

        sorted_results = sorted(filtered_results, key=lambda x: x[3])
    
        sorted_results = [result[1:] for result in sorted_results]

        return sorted_results
        
        cur.close()
        connection.close()

    except psycopg2.DatabaseError as e:
        print(f"Error: {str(e)}")
        return []


ranked_results_via_ld = levenshtein_dist_from_db(csv_name, pdf_data_points)
relevant_columns_from_pdf = [col.strip() for col in pdf_relevant_columns.split(",")]
validated_data_points_via_ld = []

for col in relevant_columns_from_pdf:
    for data in ranked_results_via_ld:
        if data[0] == col:
            data_str = "( Column Name: " + str(data[0]) + ", Value: " + str(data[1]) + " )"
            validated_data_points_via_ld.append(data_str)


validated_data_points_via_ld = " ".join(str(element) for element in validated_data_points_via_ld[:5])
rprint(validated_data_points_via_ld)



( Column Name: zipcode, Value: 11222 ) ( Column Name: zipcode, Value: 11223 ) ( Column Name: zipcode, Value: 11225 
) ( Column Name: zipcode, Value: 11226 ) ( Column Name: zipcode, Value: 10024 )

In [48]:
from langchain_postgres import PGVector


def semantic_search(table_name:str, words: str):
    """distance between vectors"""

    collection_name = table_name + "_collection"

    embeddings = OpenAIEmbeddings(
        openai_api_key=OPENAI_API_KEY,
        openai_api_base=OPENAI_BASE_URL,
        model=OPENAI_EMB_MODEL_SMALL,
        dimensions=512
    )
    
    vector_store = PGVector(
        embeddings=embeddings,
        collection_name=collection_name,
        connection=db_url,
    )

    results = vector_store.similarity_search_with_score(query=words, k=3)

    return results

ranked_results_via_similarity = semantic_search("google_maps_restaurants_shortened_2", pdf_data_points)

validated_data_points_via_similarity = []

for doc, score in ranked_results_via_similarity:
    for col_name in doc.metadata:
        if col_name in relevant_columns_from_pdf:
            data_str = "( Column Name: " + str(col_name) + ", Value: " + str(doc.metadata[col_name]) + " )"
            validated_data_points_via_similarity.append(data_str)

validated_data_points_via_similarity = " ".join(str(element) for element in validated_data_points_via_similarity)
rprint(validated_data_points_via_similarity)

( Column Name: address, Value: 18 Bedford Ave, Brooklyn, NY 11222 ) ( Column Name: zipcode, Value: 11222 ) ( Column
Name: address, Value: 5 Martense St, Brooklyn, NY 11226 ) ( Column Name: zipcode, Value: 11226 ) ( Column Name: 
address, Value: 1953-1955 Coney Island Ave floor #1, Brooklyn, NY 11223 ) ( Column Name: zipcode, Value: 11223 )

In [49]:
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


inputs = { "question": "Whats a good restaurant near NY Aquarium?" , "pdf_data": pdf_answer, "table_data": validated_data_points_via_ld + validated_data_points_via_similarity }

class DataAnalystResponse(BaseModel):
    """Model for the data analyst's JSON response."""
    augmented_question: str = Field( description="The augmented question with data points from the table and pdf")
    table_data_points: str = Field(description="Data points from the table that make the question more specific.")
    query_type: str = Field(description="Either 'retrieval' or 'manipulation'")

def create_data_analyst_prompt(format_instructions):
    prompt_template = PromptTemplate(
    template="""
            system,
            You are a data analyst that deciphers the right information to augment the initial question.
            Your goal is to augment the question with data points from the table so that a SQL query can be created to answer the question.

            You are given the following information:
            The initial question is:
            {question}

            Information from a pdf_data is:
            {pdf_data}

            The table_data contains column names and their respective values. Here is the table_data:
            {table_data}

           Steps:
            1. Augment the question with data from the table_data and the pdf_data to make the question more specific so that a SQL query can be created. 
            2. Use the relevant data points in table_data to enhance the augmented question. Disregard data that strays form the question.
            3. Decipher if the question would require a Modify Data Query (e.g., Insert, Update, Delete, Merge, Create, Alter, Drop, Truncate, Rename, etc) or a Retrieve Query (e.g., Select, Group By, Order By, etc.)

            Follow the ouptut schema below:
            {format_instructions}

            """,
                input_variables=["question", "pdf_data", "table_data"],

                partial_variables={"format_instructions": format_instructions},
            )

    return prompt_template


In [52]:
def json_parser_prompt_chain_data_analyst(inputs):
    """Allow user to ask a question to the model and get a json response. User can change the model."""
    parser = JsonOutputParser(pydantic_object=DataAnalystResponse)
    prompt = create_data_analyst_prompt(format_instructions=parser.get_format_instructions())
    chain = prompt | model | parser
    response = chain.invoke(inputs)
    return response

augment_question_llm_response = json_parser_prompt_chain_data_analyst(inputs)
rprint(augment_question_llm_response)

{
    'augmented_question': "What's a good restaurant near the NY Aquarium located at 602 Surf Avenue, Brooklyn, New 
York 11224?",
    'table_data_points': 'Zipcodes near NY Aquarium: 11222, 11223, 11225, 11226. Addresses of restaurants: 18 
Bedford Ave, Brooklyn, NY 11222; 5 Martense St, Brooklyn, NY 11226; 1953-1955 Coney Island Ave floor #1, Brooklyn, 
NY 11223.',
    'query_type': 'retrieval'
}

In [54]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits import create_sql_agent
from pgvector.sqlalchemy import Vector



def create_sql_multiagent_retrieval_prompt(question: str, table_data_points: str):
    prompt_template = f"""
                    You are a SQL specialist who can write SQL queries to answer the user's question. 
                    You must produce two queries:
                    1) An **answer_retrieval_query query** that directly addresses the user's question.
                    2) A **visualization query** to help visualize the results of the answer query.

                    The user's question: {question}
                    Valid Data points from the table (column & value): {table_data_points}

                    Requirements for the answer_retrieval_query:
                    - Valid data points are curated from the table data to help you avoid mistakes in your query, use them to correct your query.
                    - If the answer_retrieval_query does not yield any results or fails, then set "next_agent" to "human_input". Explain why it failed in the "answer" field, provide suggestions, and wait for user response.
                    - If successful and results are more than 0, place the final tested query in "answer_retrieval_query". 
                    - Do not use "LIMIT" unless needed to answer the question.

                    Requirements for the visualization query:
                    - Include ctids in the SELECT clause.
                    - Only provide a visualization query if the answer_retrieval_query returns results (non-empty).
                    - The query should be specific to the rows that answer the question.
                    - If aggregation is needed, ensure all non-aggregated columns are in the GROUP BY clause.
                    - Typically select the whole row (ctid, *), or select just ctid plus relevant columns.

                    Label the visualization query (max 7 words), e.g. "Select all cars", "Select running totals", etc.

                    Return the following JSON exactly (fill in the fields accordingly, no extra fields):

                    {{
                    "answer": "<_START_> Explanation of why this query is best, or explanation of failure. <_END_>",
                    "query_type": "retrieval",
                    "answer_retrieval_query": "The final tested retrieval query",
                    "visualize_retrieval_query": "Query for visualization (if any)",
                    "visualize_retrieval_label": "A label describing the visualization query"
                    }}
                    """

    return prompt_template


def call_sql_agent(prompt:str, table_name: str):
    """Call the SQL Agent langchain toolkit."""
    db_for_table = SQLDatabase.from_uri(db_url, include_tables=[table_name])
    toolkit = SQLDatabaseToolkit(db=db_for_table, llm=model)
    sql_agent_for_table = create_sql_agent( llm=model, 
                                            toolkit=toolkit,
                                            agent_type="openai-tools",
                                            verbose=False,
                                            agent_executor_kwargs={"return_intermediate_steps": True})
    
    sql_result = sql_agent_for_table.invoke(prompt)

    return sql_result


augmented_question = augment_question_llm_response["augmented_question"]
table_data_points = augment_question_llm_response["table_data_points"]
prompt = create_sql_multiagent_retrieval_prompt(augmented_question, table_data_points)

sql_result = call_sql_agent(prompt=prompt, table_name = "google_maps_restaurants_shortened_2")
rprint(sql_result)



{
    'input': '\n                    You are a SQL specialist who can write SQL queries to answer the user\'s 
question. \n                    You must produce two queries:\n                    1) An **answer_retrieval_query 
query** that directly addresses the user\'s question.\n                    2) A **visualization query** to help 
visualize the results of the answer query.\n\n                    The user\'s question: What\'s a good restaurant 
near the NY Aquarium located at 602 Surf Avenue, Brooklyn, New York 11224?\n                    Valid Data points 
from the table (column & value): Zipcodes near NY Aquarium: 11222, 11223, 11225, 11226. Addresses of restaurants: 
18 Bedford Ave, Brooklyn, NY 11222; 5 Martense St, Brooklyn, NY 11226; 1953-1955 Coney Island Ave floor #1, 
Brooklyn, NY 11223.\n\n                    Requirements for the answer_retrieval_query:\n                    - 
Valid data points are curated from the table data to help you avoid mistakes in your query, use them to correct 
your query.\n                    - If the answer_retrieval_query does not yield any results or fails, then set 
"next_agent" to "human_input". Explain why it failed in the "answer" field, provide suggestions, and wait for user 
response.\n                    - If successful and results are more than 0, place the final tested query in 
"answer_retrieval_query". \n                    - Do not use "LIMIT" unless needed to answer the question.\n\n     
Requirements for the visualization query:\n                    - Include ctids in the SELECT clause.\n             
- Only provide a visualization query if the answer_retrieval_query returns results (non-empty).\n                  
- The query should be specific to the rows that answer the question.\n                    - If aggregation is 
needed, ensure all non-aggregated columns are in the GROUP BY clause.\n                    - Typically select the 
whole row (ctid, *), or select just ctid plus relevant columns.\n\n                    Label the visualization 
query (max 7 words), e.g. "Select all cars", "Select running totals", etc.\n\n                    Return the 
following JSON exactly (fill in the fields accordingly, no extra fields):\n\n                    {\n               
"answer": "<_START_> Explanation of why this query is best, or explanation of failure. <_END_>",\n                 
"query_type": "retrieval",\n                    "answer_retrieval_query": "The final tested retrieval query",\n    
"visualize_retrieval_query": "Query for visualization (if any)",\n                    "visualize_retrieval_label": 
"A label describing the visualization query"\n                    }\n                    ',
    'output': '{\n"answer": "<_START_> The query successfully retrieved restaurants near the NY Aquarium by 
filtering for those within the relevant zip codes (11222, 11223, 11225, 11226). This approach ensures that the 
results are geographically relevant to the user\'s request for a good restaurant near the NY Aquarium. Among the 
results, \'Noula Restaurant\' with a rating of 4.9 stands out as a highly rated option. <_END_>",\n"query_type": 
"retrieval",\n"answer_retrieval_query": "SELECT id, name, rating, address, zipcode FROM 
google_maps_restaurants_shortened_2 WHERE zipcode IN (11222, 11223, 11225, 11226)",\n"visualize_retrieval_query": 
"SELECT ctid, id, name, rating, address, zipcode FROM google_maps_restaurants_shortened_2 WHERE zipcode IN (11222, 
11223, 11225, 11226)",\n"visualize_retrieval_label": "Visualize Nearby Restaurants"\n}',
    'intermediate_steps': [
        (
            ToolAgentAction(
                tool='sql_db_list_tables',
                tool_input={},
                log='\nInvoking: `sql_db_list_tables` with `{}`\n\n\n',
                message_log=[
                    AIMessageChunk(
                        content='',
                        additional_kwargs={
                            'tool_calls': [
                               